In [1]:
import json
from pathlib import Path

### With Dicts
Encoding from and decoding to `dict`s is easy. Just use the `json.loads` and `json.dumps` or their file obj counterparts `json.load` and `json.dump`

In [2]:
cookies = [
    {"flavor": "Chocolate Chip", "calories": 180},
    {"flavor": "Snicker Doodle", "calories": 220},
    {"flavor": "Oatmeal Raisin", "calories": 120},
]

print(json.dumps(cookies, indent=2))

with open("cookies.json", "wt") as fout:
    json.dump(cookies, fout)

Path("./cookies.json").exists() 

[
  {
    "flavor": "Chocolate Chip",
    "calories": 180
  },
  {
    "flavor": "Snicker Doodle",
    "calories": 220
  },
  {
    "flavor": "Oatmeal Raisin",
    "calories": 120
  }
]


True

In [3]:
json_cookies = """
[
  {
    "flavor": "Chocolate Chip",
    "calories": 180
  },
  {
    "flavor": "Snicker Doodle",
    "calories": 220
  },
  {
    "flavor": "Oatmeal Raisin",
    "calories": 120
  }
]
"""
json.loads(json_cookies)

with open("cookies.json", "rt") as fin:
    cookies = json.load(fin)
print(cookies)

[{'flavor': 'Chocolate Chip', 'calories': 180}, {'flavor': 'Snicker Doodle', 'calories': 220}, {'flavor': 'Oatmeal Raisin', 'calories': 120}]


However the default encoder/decoder only supports this [conversion table](https://docs.python.org/3/library/json.html#py-to-json-table)

| Python | JSON |
|--------|------|
| dict | object |
| list, tuple | array |
| str | string |
| int, float, int- & float-derived Enums | number |
| True | true |
| False | false |
| None | null |

Anything that is not in this table will raise a `TypeError` when encoding. E.g., if my data has a `datetime` object, it is cannot be encoded to JSON. Conversely if my json string has a datetime string that I want to convert to `datetime` object in my resulting `dict`, I cannot do that. Another problem is that I am stuck with the default encoding/decoding, e.g., if I want to change the key names or want the schema to not mirror my dict structure, I cannot do that.

There are two different solutions to these problems - one for encoding and one for decoding.

### Encoding
The solution is to implement a custom `JSONEncoder` and implement the `default()` method.

In [4]:
from dataclasses import dataclass
from datetime import date, datetime
from json import JSONEncoder
from typing import Any


@dataclass
class Book:
    title: str
    published_on: date


class BookJSONEncoder(JSONEncoder):
    def default(self, book: Book) -> dict[str, Any]:
        return {
            "title": book.title,
            "published_on": datetime.strftime(book.published_on, "%Y-%m-%d")
        }
    

books = [
    Book(
        title="Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones", 
        published_on=datetime.strptime("16 October 2018", "%d %B %Y").date()
    ),
    Book(
        title="The Dip: The extraordinary benefits of knowing when to quit",
        published_on=datetime.strptime("01 January 2007", "%d %B %Y").date()
    )
]

print(json.dumps(books, indent=2, cls=BookJSONEncoder))

[
  {
    "title": "Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones",
    "published_on": "2018-10-16"
  },
  {
    "title": "The Dip: The extraordinary benefits of knowing when to quit",
    "published_on": "2007-01-01"
  }
]


### Decoding
The solution is to use the `object_hook` or the `object_pairs_hook` callbacks in the `json.load` API.

In [5]:
def decode_book(book_dict: dict[str, Any]) -> Book:
    return Book(
        title=book_dict["title"],
        published_on=datetime.strptime(book_dict["published_on"], "%Y-%m-%d").date()
    )


book_json = """
[
  {
    "title": "Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones",
    "published_on": "2018-10-16"
  },
  {
    "title": "The Dip: The extraordinary benefits of knowing when to quit",
    "published_on": "2007-01-01"
  }
]
"""

books = json.loads(book_json, object_hook=decode_book)
print(books)

[Book(title='Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones', published_on=datetime.date(2018, 10, 16)), Book(title='The Dip: The extraordinary benefits of knowing when to quit', published_on=datetime.date(2007, 1, 1))]
